# [手把手教你完成一个数据科学小项目（1）：数据爬取](https://zhuanlan.zhihu.com/p/42060094) 
请先阅读[“中国年轻人正带领国家走向危机”，这锅背是不背？](https://zhuanlan.zhihu.com/p/41880873) 一文，以对“手把手教你完成一个数据科学小项目”系列有个全局性的了解。

偶然间在微博看到有人分享了这篇文章，好奇对于“年轻人正带领国家走向危机”的言论，大家的反应和舆论是怎样的，于是用爬虫发现能便捷的爬取到评论数据，那就来看看能分析和挖掘出什么信息。另外刚完成了[当你点开这首《八月》的歌：9400余条评论的分析与挖掘》](https://zhuanlan.zhihu.com/p/41516392)这个小项目，也开源了数据和代码：https://github.com/DesertsX/gulius-projects ，正好也是评论数据的挖掘，所以部分代码和经验可以迁移过来。此外，介绍并安利下 [gulius-procjects](https://github.com/DesertsX/gulius-projects) 这个项目，创建的初衷是开源自己完成过的小项目的代码，主要是爬虫、数据分析与挖掘、可视化等用 Python 的全套实现，尽量不手动用 excel 等辅助，算是磨练下代码能力；另外也是将一些“野生”数据分析、挖掘、可视化的套路与模板整合起来，作为“武器库”方便日后迁移与应用。更多文章详情见于知乎专栏[《Data Analysis & Viz》 用可视化看见不一样的内容](https://zhuanlan.zhihu.com/c_122608198)。

## 抓包一下，你就知道
言回正传。 

决定本次项目能否推进的一个关键因素就是能否爬取到文章底下评论的数据。虽然专栏里曾写过不少基于爬虫爬取的数据进行挖掘和可视化的文章，但因长久没写过爬虫，很是生疏了，且不敢托大说看到感兴趣的数据就一定能爬下来，因此向来都是先在网页的“审查元素”与抓包过程、及测试代码中不断评估难易程度。

本回进行多次翻页并观察抓包后得到的 API ，可知仅 `page=4` 和 `jsvar=loader_1533711754393_51961031` 两处发生变化，前者是评论的页数，后者13位的数字`1533711768432`与8位数字`76083893`看似随机，但有些经验的朋友应该也能猜到是时间戳，因而猜想是翻页过程时读取数据的起始和结束的时间点:
- http://comment5.news.sina.com.cn/page/info?version=1&format=js&channel=cj&newsid=comos-hhkuskt2879316&group=0&compress=0&ie=gbk&oe=gbk&page=4&page_size=20&jsvar=loader_1533711768432_76083893
- http://comment5.news.sina.com.cn/page/info?version=1&format=js&channel=cj&newsid=comos-hhkuskt2879316&group=0&compress=0&ie=gbk&oe=gbk&page=3&page_size=20&jsvar=loader_1533711754393_51961031 

也是因为刚之前爬取网易云音乐歌曲评论（[gulius-projects/1_Netease_Music_Comment_Spider](https://github.com/DesertsX/gulius-projects/tree/master/1_Netease_Music_Comment_Spider)）时也遇到了13位的时间戳，因此这回“确认过眼神”，一定没错。以下是验证步骤，可以看到和 jsvar 里的数字形式很相似。

In [7]:
import time
commentTime = 1533711754393 # 13 位 # 一般是 10 位
print(commentTime)
print(int(time.time()))
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(commentTime/1000))) # 将10位的时间戳转换成一般的时间格式

1533711754393
1533712827
2018-08-08 15:02:34


构造 url，并用 requests 库尝试发送 get 请求，发现不用带上其它参数，就已经能拿到数据，简直轻松。

In [17]:
import requests
import time

# jsvar=loader_1533711754393_51961031
for page in range(0,150):
    start = int(time.time()*1000)
    end_stamp = start + random.randint(100,1000) # 随机构造时间间隔，并加到起始时间戳上
    end = str(end_stamp)[-8:]
    url = 'http://comment5.news.sina.com.cn/page/info?version=1&format=js&channel=cj&newsid=comos-hhkuskt2879316&group=0&compress=0&ie=gbk&oe=gbk&page={}&page_size=20&jsvar=loader_{}_51961031'.format(page, start, end)
    r = requests.get(url).text
    print(r)
    break # 测试步骤，爬一两页看看能否拿到数据，以及数据格式如何

var loader_1533713810791_51961031={"result": {"status": {"msg": "", "code": 0}, "count": {"qreply": 9526, "total": 18107, "show": 2995}, "replydict": {"5B6A8A1D-AFABADB5-13D26DD6E-8C5-928": [{"comment_imgs": "", "parent_mid": "0", "news_mid_source": "0", "rank": "0", "mid": "5B69ABE9-6518CCA4-70D4B53F-8C5-87A", "video": "", "vote": "0", "uid": "1892988223", "area": "\u6cb3\u5317\u77f3\u5bb6\u5e84", "channel_source": "", "content": "\u51e0\u5e74\u524d\u6709\u90a3\u4e48\u4e00\u6279\u4eba\u8fd8\u770b\u4e0d\u8d77\u7f8e\u56fd\u5e74\u8f7b\u4eba\uff0c\u8bf4\u7f8e\u56fd\u5e74\u8f7b\u4eba\u90fd\u662f\u501f\u94b1\u751f\u6d3b\uff0c\u53ef\u803b\u3002\u8fd9\u624d\u51e0\u5e74", "nick": "\u4e00\u5b9a\u8981\u675c\u7edd", "hot": "0", "status_uid": "1627825392", "content_ext": "", "ip": "101.24.204.164", "media_type": "0", "config": "wb_verified=0&wb_screen_name=\u4e00\u5b9a\u8981\u675c\u7edd&wb_cmnt_type=comment_retweeted_status&wb_user_id=1892988223&wb_description=&area=\u6cb3\u5317\u77f3\u5bb6\u5e84&

var loader_1533713812067_51961031={"result": {"status": {"msg": "", "code": 0}, "count": {"qreply": 9526, "total": 18107, "show": 2995}, "replydict": {"5B6A98A9-3AD27E12-87E2BF1F-8C5-84F": [{"comment_imgs": "", "parent_mid": "0", "news_mid_source": "0", "rank": "24", "mid": "5B6A5FA2-7A050D7C-BC906B4E-8C5-897", "video": "", "vote": "0", "uid": "3163581262", "area": "\u5c71\u4e1c\u70df\u53f0", "channel_source": "", "content": "\u6211\u53ef\u771f\u725b\u903c \u6211\u8fd8\u60f3\u5e26\u9886\u4eba\u7c7b\u8d70\u5411\u6bc1\u706d", "nick": "\u5e26\u5e26\u5e26\u8868\u54e5\u5e26\u4f60\u53bb\u6d6a\u6f2b\u571f\u8033\u5176", "hot": "0", "status_uid": "1638782947", "content_ext": "", "ip": "122.5.13.124", "media_type": "0", "config": "wb_verified=0&wb_screen_name=\u5e26\u5e26\u5e26\u8868\u54e5\u5e26\u4f60\u53bb\u6d6a\u6f2b\u571f\u8033\u5176&wb_cmnt_type=comment_status&wb_user_id=3163581262&wb_description=&area=\u5c71\u4e1c\u70df\u53f0&wb_parent=&wb_profile_img=http%3A%2F%2Ftvax3.sinaimg.cn%2Fcrop.0.

截取掉返回的数据前面的多余部分后，就是`'{ "result":{status: {msg: "", code: 0}, count: {qreply: 8948, total: 17341, show: 2902}, replydict: {,…},…}'`这样的看起来像字典，实则是字典样格式的字符串。可用 eval() 将字典样、列表样的字符串转换成字典或列表。

In [16]:
s='var loader_1533713674946_51961031='
print(len(s))

34


突然想到 eval() 和 json.loads() 都能讲字典样的字符串转换成字典，那两者区别是什么呢？搜了下，诸位感兴趣的可以一看：[eval 与 json.loads 对 json 的处理小记](https://blog.csdn.net/djskl/article/details/44783777)  

接下来，截取掉 `requests.get(url).text` 的前面字符，并测试看看能否正常拿到数据。

In [29]:
import time
import requests
import json

# jsvar=loader_1533711754393_51961031
for page in range(0,150):
    start = int(time.time()*1000)
    end_stamp = start + random.randint(100,1000)
    end = str(end_stamp)[-8:]
    url = 'http://comment5.news.sina.com.cn/page/info?version=1&format=js&channel=cj&newsid=comos-hhkuskt2879316&group=0&compress=0&ie=gbk&oe=gbk&page={}&page_size=20&jsvar=loader_{}_51961031'.format(page, start, end)
    r = requests.get(url).text
    json_dict = json.loads(r[34:])
    cmntlist = json_dict['result']["cmntlist"]
    for num,cmnt in  enumerate(cmntlist):
        print(num, cmnt['nick'],cmnt['area'], cmnt['time'], cmnt['content']) # 测试看看能否正常拿到数据
    break

0 用户957zwswd 2018-08-08 14:21:26 这是什么沙雕 浙江嘉兴
1 _尾尾_ 2018-08-08 14:21:25 有啊，税制改革了，有学生、老人的家庭有税收优惠，也就意味着，单身要交更多的税了。 宁夏银川
2 可乐味的我想和你天下第一好 2018-08-08 14:21:24 那叫探索，叫探索时的必经之路[作揖] 河南洛阳
3 阳先生53719 2018-08-08 14:21:20 [二哈]不是说你，而是你转发的最右的博主 江苏苏州
4 姗妹· 2018-08-08 14:20:49 呸 我买不起房还不让买包吗@皮卡丘究极进化 安徽马鞍山
5 三岔丶路口 2018-08-08 14:20:48 父母辛辛苦苦工作一辈子也凑不够首付，怪父母啥？怪他们生了孩子？？ 河北保定
6 killjapan1945815 2018-08-08 14:20:41 我这老人家还真不知道 香港
7 一弥猫一 2018-08-08 14:20:25 说的好[doge] 广东深圳
8 killjapan1945815 2018-08-08 14:20:06 我是打比方[笑cry] 香港
9 SmakCheng 2018-08-08 14:20:00 我感觉出家是条好出路！能干的事更多[doge] 浙江杭州
10 蓝国之恋冰咖啡 2018-08-08 14:19:45 高级黑 山东泰安
11 Erebus_Chaos 2018-08-08 14:19:33 你怎么能说出这么不要脸的话[允悲]我服了你 江苏苏州
12 小木鱼号 2018-08-08 14:19:33 心累，上个学小费万数起，还到处求人帮忙。 浙江温州
13 吃瓜阿声 2018-08-08 14:19:27 你不是很自豪吗 广东深圳
14 微笑的零 2018-08-08 14:19:24 牛逼 广东深圳
15 国士无双三师公 2018-08-08 14:19:21 天灾飞锅！怎么都躲不掉 四川成都
16 没有现充的欧阳666 2018-08-08 14:19:18 ？？？年轻人背锅？ 美国乔治亚
17 -北坡 2018-08-08 14:19:18 有点看不懂…迷… 黑龙江哈尔滨
18 黑奇士司马子羽 2018-08-08 14:19:06 。。。 北京海淀
19 舞庭前百长弓年少 2018-08-

要提取的数据都在这里了，可根据感兴趣、
想分析和挖掘的数据进行相应提取。

In [30]:
json_dict

{'result': {'cmntlist': [{'against': '1533709286',
    'agree': '0',
    'area': '浙江嘉兴',
    'audio': '',
    'channel': 'cj',
    'channel_source': '',
    'comment_imgs': '',
    'comment_mid': '0',
    'config': 'wb_verified=0&wb_screen_name=用户957zwswd&wb_cmnt_type=comment_status&wb_user_id=3485136660&wb_description=&area=浙江嘉兴&wb_parent=&wb_profile_img=http%3A%2F%2Ftvax2.sinaimg.cn%2Fcrop.0.0.1125.1125.50%2Fcfbaf714ly8fsk2s00ft5j20v90v9n0k.jpg&wb_time=2018-08-08 14:21:12&wb_comment_id=4270748586173315',
    'content': '这是什么沙雕',
    'content_ext': '',
    'hot': '0',
    'ip': '122.238.190.216',
    'length': '6',
    'level': '0',
    'login_type': '0',
    'media_type': '0',
    'mid': '5B6A8BE2-7AEEBED8-CFBAF714-8C5-8CA',
    'news_mid': '0',
    'news_mid_source': '0',
    'newsid': 'comos-hhkuskt2879316',
    'newsid_source': '',
    'nick': '用户957zwswd',
    'openid': '',
    'parent': '',
    'parent_mid': '0',
    'parent_nick': '',
    'parent_profile_img': '',
    'parent_u

## 爬虫走你~
经过上述步骤，大致确定能正常爬取到所需要的数据了。可能有经验的朋友会觉得上面都很简单啊，怎么讲了那么久，似乎没有必要，直接把爬虫完整代码贴上，自己就能理解的。但我自己的考虑是尽量把代码缘由讲清楚，一些测试的过程也讲下，方便新手小白的朋友了解我在弄这个项目时的思路和过程，若觉得有所启发，也就很欣慰了。本项目包含后续内容也算是**手把手教大家怎么完成一个小的“数据科学”小项目了吧**（打“数据科学”四字时有些虚，膨胀了，数据科学都敢扯了），不过也和本项目初衷相符：[gulius-procjects](https://github.com/DesertsX/gulius-projects)
<img src='http://img.hb.aicdn.com/c9dded206bd29759c9bdb2533b0085910a5eaab51351-NjaRgO_fw658'>

接下来完善和优化代码，正式开始爬取数据。

In [5]:
%% time 
# 计算耗时
import requests
import time
import random
import json
import pandas as pd

comments = pd.DataFrame(columns = ['page','jsons','cmntlist','replydict'])
start_page = 0 # 修改起始页数（初始值为 0）和 csv 文件名，方便爬虫中断后，继续爬取，之后再将几个 csv 数据整合到一起即可 # 断点续爬

# http://comment5.news.sina.com.cn/page/info?version=1&format=js&channel=cj&newsid=comos-hhkuskt2879316&group=0&compress=0&ie=gbk&oe=gbk&page=3&page_size=20&jsvar=loader_1533711754393_51961031
# jsvar=loader_1533711754393_51961031
try:
    # 手动设置需要爬取的总页数，评论数若有3000条，那每页20条，就设为150，当然一般在爬取时也可能有新增的评论，所以设大些更好。
    for page in range(start_page,200):  
        start = int(time.time()*1000)
        end_stamp = start + random.randint(100,1000)
        end = str(end_stamp)[-8:] # # jsvar=loader_1533711754393_51961031
        url = 'http://comment5.news.sina.com.cn/page/info?version=1&format=js&channel=cj&newsid=comos-hhkuskt2879316&group=0&compress=0&ie=gbk&oe=gbk&page={}&page_size=20&jsvar=loader_{}_51961031'.format(page, start, end)
        content = requests.get(url).text
        jsons = json.loads(content[34:])  # var loader_1533713810791_51961031= { "result":{status: {msg: "", code: 0}, count: {qreply: 8948, total: 17341, show: 2902}, replydict: {,…},…}
        cmntlist = jsons['result']["cmntlist"]
        replydict = jsons['result']["replydict"]
        # jsons 里有几乎所有数据，方便后续爬虫结束也能本地进行提取 # 不过本次主要对 jsons 里的 cmntlist 和 replydict 感兴趣，所以也先提取了
        comments = comments.append({'page':page+1,'jsons':jsons,'cmntlist':cmntlist,'replydict':replydict},ignore_index=True)
        for num,cmnt in  enumerate(cmntlist):
            print(page*20+num+1, page+1, cmnt['nick'], cmnt['time'], cmnt['content'], cmnt['area'])
        if page%5==0:
            time.sleep(random.randint(0,2)) # 每5页随机停0-2秒，作为简单的防反爬的一步
except:
    print("Error")
comments.to_csv('Sina_Finance_Comments_1_20180811.csv',index=False,encoding='utf-8')

1 1 用户6525940293 2018-08-09 07:26:04 贷款买房你怎么说？[二哈][二哈] 江苏南京
2 1 为什么要开网店 2018-08-09 07:23:40 我们在高额贷款利息下 能活着就不错了 辽宁铁岭
3 1 了了了了丶 2018-08-09 07:22:41 有链接吗[doge] 湖南怀化
4 1 张儁乂 2018-08-09 07:12:42 你家庭杠杆加到几多？ 广东广州
5 1 用户6618549650 2018-08-09 06:54:49 胡说八道 江苏无锡
6 1 不一般的杜 2018-08-09 06:21:10 别操心了，早睡早起吧 北京
7 1 百利今天胖了吗 2018-08-09 06:20:04 这里说的是借钱啊 福建厦门
8 1 百利今天胖了吗 2018-08-09 06:18:30 这里说的是借钱消费啊，自己赚的当然无所谓 福建厦门
9 1 icecream1013 2018-08-09 06:16:39 人在家中躺，锅从天上来😂 北京
10 1 佐佐木伊之助 2018-08-09 06:12:08 ？年轻人得罪谁了？@瞄头打爆蛋  @举头三尺有神麟  @v白白的黑子 山东济宁
11 1 上善若水9220 2018-08-09 06:11:01 不好意思，上次我睡着了，没回复你的信息。祝你在上海一切顺利.下次来北京了，要挤点时间[二哈][笑cry] 北京
12 1 全世界最好的Tao小狗 2018-08-09 06:02:25 哈哈哈哈哈 穷人没有选择[跪了] 英国英格兰
13 1 蝴蝶_喵喵喵 2018-08-09 05:50:26 新周刊好刚！ 四川成都
14 1 Sugarpppppp 2018-08-09 05:32:44 完全没有想法买不动产啥的…就有钱就花喽，开心就好呀 江苏泰州
15 1 Sugarpppppp 2018-08-09 05:32:10 emmmm我是那种yolo族，人应该享受人生，即使需要承担风险。就是鼓励人们不怕冒险，想做什么就做什么，享受人生，因为只会活一次。 江苏泰州
16 1 你有事吗0 2018-08-09 05:08:40 我可真谢谢您了......还是头一回听说这么荒谬的理论 四川成都
17 1 hksyf 2018-08-09 03:50:33 心情复杂 江苏无锡

161 9 永一的马甲 2018-08-09 09:13:41 要的要的[允悲] 北京
162 9 Super大佩佩 2018-08-09 09:13:40 哪个年轻人不靠借贷生活！谁不欠银行上百万房贷！！！不借贷买不起啊！！！怎么不去谴责一下那些让年轻人负债累累失去梦想的恶势力！！！ 贵州贵阳
163 9 睡一夜就馊星人 2018-08-09 09:10:40 哈哈，昨天扬州媒体鼓吹幼儿园“减负”，嘿，真没想到，一大堆家长赞同……果然人和人不一样啊…… 江苏南京
164 9 李鸡肋 2018-08-09 09:04:20 老是写这些无脑文章怕是故意找骂吧 广东广州
165 9 SKY-蓝天 2018-08-09 09:03:44 还要脸不？ 广东广州
166 9 Qing诗卿 2018-08-09 09:02:00 不是说贬值吗？存美元？ 云南昆明
167 9 Max的旧货店 2018-08-09 09:01:16 真是恶心 安徽合肥
168 9 垃圾筐与电影院 2018-08-09 08:55:27 有个研究显示，70后智商是持续下降的。 四川成都
169 9 刚马甲 2018-08-09 08:49:28 呵 商品经济炒作挖坑哄年轻人消费生孩子还不是你们上一辈正在做的 上海
170 9 无双199207 2018-08-09 08:43:59 10万是一平米到0.4平米，谢谢。 河北承德
171 9 大着肚子做B超 2018-08-09 08:42:45 哈哈哈。把锅甩给年轻人了。厉害 江苏常州
172 9 amie是佳妹儿 2018-08-09 08:41:21 但直觉告诉我之后钱会越来越是钱，勒紧裤腰带多存点吧 四川成都
173 9 Just-sosoul 2018-08-09 08:37:36 我是来看评论的 河南开封
174 9 5块二毛1 2018-08-09 08:37:16 我们腰杆硬，还能挺挺 四川乐山
175 9 陶慕白zz 2018-08-09 08:35:20 我去年买了个登山包超耐磨[二哈][二哈][二哈] 北京
176 9 Nomine_noBB 2018-08-09 08:33:07 不➕[doge]或引号？ 湖北武汉
177 9 群租房mvp 2018-08-09 08:32:03 当初不是你们让我们掏空3个钱包买房的？[费解][

301 16 各位观众四条诶 2018-08-08 21:45:20 下个id见 江苏徐州
302 16 优雅妈20130114 2018-08-08 21:44:55 呵呵 北京
303 16 VVV-N南南 2018-08-08 21:42:49 已举报😌😌😌 浙江温州
304 16 古风陈韵 2018-08-08 21:35:22 呵呵呵……还没到年龄吧！ 陕西西安
305 16 梅婼千然 2018-08-08 21:33:24 让你自己挖坑自己跳，挖的越深埋的越瓷实[吃瓜] 山东聊城
306 16 S阿洁编不出来瞎话 2018-08-08 21:33:23 哈哈哈哈哈哈哈 上海
307 16 心在飞翔的家 2018-08-08 21:32:35 不把学校社保医疗绑在房子上，房子还有多大的价值呢？ 福建福州
308 16 温州大甜心_ 2018-08-08 21:28:36 Emmmm其实 中年人也在借贷 浙江
309 16 BAEL-X 2018-08-08 21:27:14 [吃瓜][吃瓜][吃瓜] 山西太原
310 16 旺旺旺甜甜 2018-08-08 21:26:19 我们这届年轻人真的好倒霉 河南郑州
311 16 幸运的艾米气色好 2018-08-08 21:25:32 感觉自己超厉害 带领全中国 美国俄亥俄州
312 16 St亓_还得再咸个一年_曜零 2018-08-08 21:22:44 我甚至不知道举报时选不实信息还是人身攻击[并不简单][并不简单][并不简单][并不简单][并不简单] 江苏徐州
313 16 天空与风先生 2018-08-08 21:22:07 你问问我们这样借贷的人渣是不是不用生二胎 广东深圳
314 16 聂公子_ 2018-08-08 21:21:52 刚才的热评第一直接被销号了？[衰] 山东泰安
315 16 奕瑾好想养只喵 2018-08-08 21:19:59 真的！好大一口锅[喵喵] 新加坡
316 16 程子庸 2018-08-08 21:19:45 炸裂！ 湖北武汉
317 16 霹雳狂刀01 2018-08-08 21:18:40 我算中年人了，这锅不用背[中国赞] 河南漯河
318 16 JK_米 2018-08-08 21:18:23 买不起房这不多的钱留着还不能让人改善一下生活了[允悲] 陕西西

601 31 huajingbaobao 2018-08-08 16:33:52 实名支持某/da/da上冥/币,整天xjb整没用的就不说了,甩锅太恶心 广东佛山
602 31 nagaoto 2018-08-08 16:33:34 就那点钱也不知道是够买厕所还是厨房的 浙江宁波
603 31 夏天来了还不减肥 2018-08-08 16:33:33 [吃瓜]我身上最大的债务就是一个名叫房贷的东西 湖北武汉
604 31 PeaceSUElove 2018-08-08 16:33:00 擦…嚯嚯差不多了该找接盘侠了[挖鼻] 黑龙江哈尔滨
605 31 憋说话表哥来了 2018-08-08 16:32:54 [喵喵]有多少花多少，但我不去借[喵喵] 广东佛山
606 31 YL王 2018-08-08 16:32:33 我18岁的时候报章天天讲勤劳致富，万元户…… 江苏徐州
607 31 八荒六合唯孤独尊 2018-08-08 16:32:27 主流的媒体舆论引导孩子的三观，而很多的无良媒体影响着孩子的性格，下一代都快被毁了，世界观、人生观、价值观都被扭曲，抗日神剧、提倡打着环保幌子为自己谋求利益的动画、没有下限的小说… 黑龙江哈尔滨
608 31 快嗦我是不是你的小仙侣 2018-08-08 16:32:25 笑死了用部分人群就直接代表中国年轻人了，那我不当年轻人当老年人好了 广西南宁
609 31 软恩RYAN 2018-08-08 16:32:05 谬赞了，不敢当 北京
610 31 五号公园的小子 2018-08-08 16:31:38 那这么说，我四条腿都被打断了？ 浙江
611 31 bh啊哈嗯哼啊哈嗯哼啊哈嗯哼 2018-08-08 16:31:02 太恶心了！ 河南
612 31 GK工坊 2018-08-08 16:30:46 等等，日常开销怎么产生债务？除了房贷和奢侈品，正常人很难产生债务的吧？房贷这是ZF的锅啊（奢侈品是社会风气的锅 吉林
613 31 劉琨童鞋 2018-08-08 16:30:37 所以嘛，年轻人关注菜头叔容易从历史观看问题[羞嗒嗒] 内蒙古呼和浩特
614 31 老徐爱吃糖 2018-08-08 16:30:23 攒钱为什么就为了买房？自己过得开心不是最重要？？ 江苏扬州
615 31 一位心态巨好的朋友 2018-08-0

761 39 自带消音高倍的AWM 2018-08-08 15:22:12 今天也是愤青的肥肥[偷笑][偷笑][偷笑] 河南
762 39 Bortuti_ 2018-08-08 15:22:06 哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈 江苏苏州
763 39 Winson76 2018-08-08 15:19:16 原来已经四十多了！ 广东清远
764 39 tzqyjy 2018-08-08 15:17:35 是听报章 /天天 上海
765 39 开了也是来打酱油的 2018-08-08 15:17:33 不去存钱买房，买手机买包包当然不行 湖南衡阳
766 39 rouroudun 2018-08-08 15:17:02 网友说新浪是卖锅的[笑cry] 北京
767 39 你哼哼我哈哈 2018-08-08 15:16:42 太有画面感了 山东淄博
768 39 树下布恩迪亚 2018-08-08 15:16:14 帝国主义的锅 凭什么让我背 陕西西安
769 39 Super-林夕汝婷 2018-08-08 15:15:52 哈哈哈哈哈 江苏苏州
770 39 郭晓婷是怪咖 2018-08-08 15:15:48 除了mmp也不知道该说什么了 福建泉州
771 39 三糊涂 2018-08-08 15:15:36 带领中国步入危机的是谁大家都知道，年轻人？他们有那能量吗？！ 江苏无锡
772 39 tktkpop 2018-08-08 15:15:12 烦不烦啊，就知道把锅甩给我们年轻人 浙江杭州
773 39 shn48-李富贵 2018-08-08 15:15:06 这真的是很讽刺了。。 湖北恩施土家族苗族自治州
774 39 女巫的格调 2018-08-08 15:14:56 好吧[允悲] 是我多虑了 美国加利福尼亚州
775 39 MegaJayChew 2018-08-08 15:14:53 攀比风真是越刮越大 江苏盐城
776 39 涛天王王王中王 2018-08-08 15:14:09 “他们从婴幼儿时期就习惯了父母无微不至的关怀和呵护”。真说对了，你这种人还真是打不得骂不得。既然你这么不服，那你来带领吧。 河北秦皇岛
777 39 玛丽貂貂貂 2018-08-08 15:14:09 神总结 神标题 上海
778 39 姚韭韭 2018-08-08 1

...

3001 151 午夜看见阳光 2018-08-08 09:02:19 造谣，举报了 广西南宁
3002 151 张de日常生活料理 2018-08-08 09:02:18 现在的年轻人都在标榜个性，但是，这些个性都是一样的。 河北廊坊
3003 151 炸裂少年图坦依蒙 2018-08-08 09:02:18 是的，放我去美国祸害美国吧，保证不辱使命 上海
3004 151 不确定小姐1125 2018-08-08 09:01:47 房贷是多少？心里没数？星巴克贵还是房子利息贵？消费降级成啥样了？心里没数？把锅甩给年轻人，还让它们扎紧裤腰带生二胎？疯了吧，丧心病狂 江苏泰州
3005 151 南水遗人 2018-08-08 09:01:44 还不是国家用娱乐把年轻人鼓励成这样的...[二哈]天天喊消费消费，现在不择手段地堆出信贷了，扭头就把锅甩给了年轻人[二哈] 辽宁大连
3006 151 改个什么名号 2018-08-08 09:01:12 哇塞原来我们有这么厉害啊 都能引领国家危机了 山东青岛
3007 151 进击的大圆脸 2018-08-08 09:01:06 90后又要背锅了，习惯了习惯了 内蒙古赤峰
3008 151 Marcia_turea 2018-08-08 09:00:51 手机用的华为，衣服买的快鱼，咖啡都是速溶的。我家也没矿，挣得钱还得存一部分。不用愁钱的年轻人都是少数 河北沧州
3009 151 alex的朗读者 2018-08-08 09:00:48 要垮了？ 四川成都
3010 151 广州李沛聪 2018-08-08 09:00:42 还不愿意生二胎 广东广州
3011 151 酒茶铺 2018-08-08 09:00:37 怎么不写这届10后不行，无耻 河北
3012 151 弗居室先生 2018-08-08 09:00:36 刚开始我也以为摔锅，你们进去看一下，好像身边人很多这样的 内蒙古呼和浩特
3013 151 纤纤微尘7755 2018-08-08 09:00:29 今早买的南瓜汁到了工作的地方撒了[允悲][允悲][允悲] 上海
3014 151 David_in梁垛花园巷一号 2018-08-08 09:00:06 喝星巴克的罪比工作100年才还得起的房贷还大! 上海
3015 151 小兰de胡同 2018-08-08

3161 159 良民愚石 2018-08-08 08:20:06 还能再不要脸点吗 山东潍坊
3162 159 盛世红霞520 2018-08-08 08:20:03 平民百姓一家要二代人一起才还得起房贷[允悲] 湖北宜昌
3163 159 老不正经的增阅同学 2018-08-08 08:19:45 回复 @译之俄:看不出来啊大哥[doge] 江西上饶
3164 159 可可西里的Cherokee 2018-08-08 08:19:34 喝一辈子星巴克的钱也不一定买得起房啊 上海
3165 159 阳光之恋201511 2018-08-08 08:19:12 大城市不了解。小城市有几个年轻人没房的？本地年轻人基本都有还不止一套。 江苏徐州
3166 159 梦里天空蓝 2018-08-08 08:19:11 原文里把P2P都算进来了[允悲][允悲][允悲] 浙江杭州
3167 159 潘海根 2018-08-08 08:19:04 这是因为马上很多人会无力偿还房贷，所以事先把锅甩起来吗？ 江西宜春
3168 159 九片自然 2018-08-08 08:19:03 我看售楼处很多刚毕业的 女生都在买房投资~~ 辽宁沈阳
3169 159 次次是五仙教地方保护主义小战士 2018-08-08 08:18:44 为了打造一篇爆款文，作者绞尽脑汁扣锅 北京
3170 159 网店吐槽专用小马甲 2018-08-08 08:17:46 真心买不起啊，买了就是一辈子的奴隶，结婚之前为了应付媳妇家人，市中心买了一套小一居，买了就租出去了，租金不低，在二环边上租了一个大两居，一家人住的挺好，租金还不贵。那套小房就当投资了，三年时间已经翻了两倍了，就当投资了。 河北石家庄
3171 159 掉进梦中 2018-08-08 08:17:13 dbq我不吃不喝生病不去医院不买手机不购物也得把这个房子拿下来 山西晋城
3172 159 茶没有饮光早变酸 2018-08-08 08:17:12 该来的还是来了 江西南昌
3173 159 -酸-- 2018-08-08 08:17:08 看我狗头，我的工资还够不着高消费呢 浙江
3174 159 掉进梦中 2018-08-08 08:16:55 年轻人太惨了，不仅要努力活到500岁还完房贷，还要被扣锅，还要熬夜掉头发 山西晋城
3175 

3301 166 新装腔指南 2018-08-08 07:46:09 不买房是假的吧   身边同龄人都在买房没房的也在为没房担忧 江苏苏州
3302 166 _清了 2018-08-08 07:46:09 这帽子可真够大的 广东深圳
3303 166 海边卡夫卡夫卡 2018-08-08 07:45:48 我只说一句，是谁在不遗余力的刺激人们透支消费？我相信有信用卡的人多数都收到过银行通知你临时额度可提升一倍的短信通知吧。 黑龙江哈尔滨
3304 166 Soinlife 2018-08-08 07:45:45 就是因为你不买房，所以要被说。 广东深圳
3305 166 罗健洲是我 2018-08-08 07:45:05 年轻人有这能耐? 重庆
3306 166 肖恩不是我 2018-08-08 07:44:04 别生娃了，抵制下一次危机的发起者！ 安徽合肥
3307 166 凡事看淡点 2018-08-08 07:44:04 星巴克也就是路边买咖啡的小店铺到了国内怎么就变高级了，估计和肯德基麦当劳刚出的时候一样[二哈] 浙江绍兴
3308 166 许三呆 2018-08-08 07:43:27 新一辈创富的机遇越来越少了，现在年轻人可以做什么？在我印象中，老一辈通过在工地打拼，很多人成了搞工程的老板，挣了不少钱。但很多人现在渐渐的也没落了，因为工程也不好做了，以前那种野蛮低端的挣钱方式不行了，因为文化水平不高，他们很多人现在也面临着只出不进的状态，且是多数，时代的大流。 安徽合肥
3309 166 兜兜de妈妈kiki 2018-08-08 07:43:27 看见标题就想骂人 广西南宁
3310 166 不想设昵称好麻烦 2018-08-08 07:42:32 凭什么怪我们年轻人，是这个社会发展畸形，不找主要原因，只看表面！你怎么不说买套房子我们还一辈子呢 北京
3311 166 用户6197923251 2018-08-08 07:42:15 这把倒挡已经把股市打回20年前了，难道年轻人的生活也要回到20年前吗？这样的发展，是不是为了满足某些人的心里需要啊？ 江苏苏州
3312 166 中年少女粉红兔兔兔 2018-08-08 07:42:13 这锅年轻人不背 山东济南
3313 166 呵呵哒川川 2018-08-08 07:41:33 [费解][费解][费解

3461 174 zx2194 2018-08-08 00:01:14 有用？ 上海
3462 174 伊拉克外公 2018-08-07 23:59:40 这锅帅的漂亮[赞] 广东深圳
3463 174 让小五思考下人生 2018-08-07 23:59:17 反正80后是垮掉的一代，90后是塌陷的一代，00后是需要回炉重造的一代，就你们60  70后最牛逼[吃瓜] 江西南昌
3464 174 Ver-封 2018-08-07 23:59:17 反正我是情愿喝白粥，都不去借的[允悲] 广东东莞
3465 174 王皮哦恰 2018-08-07 23:59:13 @矢周 这锅直接甩头上了 北京
3466 174 sugarba 2018-08-07 23:58:35 每一代人似乎都经历过一段被说垮掉的时光，有什么问题吗[吃瓜] 福建福州
3467 174 dear_fclbri 2018-08-07 23:57:56 儿童：我觉得是精卵结合把中国领向危机 辽宁沈阳
3468 174 彼岸-斑驳 2018-08-07 23:57:38 买不起房子，也喝不起星巴克[米奇大哭] 四川成都
3469 174 格桑雪巴 2018-08-07 23:57:33 闭门家中坐，锅从天上来。[允悲]就算有债务要你还了？[偷笑] 福建厦门
3470 174 儒-smart 2018-08-07 23:57:19 人在家中躺，锅从天上来啊😂 广东深圳
3471 174 让小五思考下人生 2018-08-07 23:56:11 80后可是垮掉的一代！当年经过多方面砖家论证过的[吃瓜] 江西南昌
3472 174 停用了的肉包仔 2018-08-07 23:56:00 那么都是在星巴克和宝宝屋的话，请问每天早高峰挤地铁上班的是谁啊？ 北京
3473 174 普罗弥修 2018-08-07 23:55:14 包括50后，一直在喷 北京
3474 174 股市股道 2018-08-07 23:54:25 标题无敌，奇天下之大葩[衰][笑cry] 云南昆明
3475 174 大菜鸟1号 2018-08-07 23:54:15 锅甩的有技术含量 湖北武汉
3476 174 包三通 2018-08-07 23:49:34 我买不起房子，你还不让我对自己好点？也不看看房子什么价格[挖鼻]买一套全家悲哀[挖

3621 182 liangliangonce 2018-08-07 22:15:36 中国年轻人这么多，您统计过过度消费的青年人数据吗，张口就来 湖南长沙
3622 182 持简易胜 2018-08-07 22:15:31 其实还是被诱导的，企业知道这钱好赚啊！ 江苏南京
3623 182 首届草民 2018-08-07 22:15:28 说这话的人长牙了嘛？[怒] 北京
3624 182 crsomething 2018-08-07 22:15:13 靠借贷生活是可耻的，那么为什么要让我们这代贷款买房[摊手]太可耻了 上海
3625 182 wiiiilllllll 2018-08-07 22:14:02 这是社会的锅 贵州
3626 182 SNH48-侯国玉 2018-08-07 22:13:57 真的 我90后从来没喷过00后 广东深圳
3627 182 gnx803 2018-08-07 22:13:57 厨子米饭做不熟，怪米不好煮？是这逻辑吧 辽宁盘锦
3628 182 独为你天荒 2018-08-07 22:13:54 钱都流向哪里了？ 重庆
3629 182 媽媽說好孩子的網名都不會取太長 2018-08-07 22:13:20 这锅我不背 上海
3630 182 手机用户3121964402 2018-08-07 22:13:14 不背不行 福建福州
3631 182 英雄6050042390 2018-08-07 22:12:03 那又怎样我就怕死了钱没花完 四川雅安
3632 182 喵酱仙人 2018-08-07 22:10:40 无耻，无耻，无耻。 福建福州
3633 182 从歪果来的歪果仁 2018-08-07 22:10:22 标题真不要脸 安徽芜湖
3634 182 no_speak 2018-08-07 22:10:20 同龄人就没有几个敢用信用卡的好吗？反而父辈一代借贷借得吓人。 广东广州
3635 182 lawrac 2018-08-07 22:08:40 醒醒，我们都不是年轻人[污] 广东广州
3636 182 Mr亚古兽 2018-08-07 22:08:06 人在家中坐，锅从天上来 山东济南
3637 182 Mr亚古兽 2018-08-07 22:07:44 人在家中坐，祸从天上来 山东济南
3638 182 守望

## 小结
爬取到数据后，就可以“为所欲为”的进行花式分析了（好吧，其实还需要进一步处理下）。大家可以读取数据，看看每行每列的格式是什么样的。

爬虫里是先按照每页20条评论的字典或列表的格式整体存储的，那么随之而来的问题就是，如何根据这些列来提取出每一条评论的数据，并存储到新的csv里呢？

可自行尝试下如何用 pandas 实现哈。更多内容将在本系列后续文章中逐步展开。

In [1]:
#comments.to_csv('Sina_Finance_Comments_1_20180811.csv',index=False,encoding='utf-8')
import pandas as pd
df = pd.read_csv('Sina_Finance_Comments_1_20180811.csv',encoding='utf-8')
df.head()

,page,jsons,cmntlist,replydict
0,1,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6B6C94-6A7ACA75-86B9ACEE-8C5-799': [{'comm...
1,2,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6C9525-6EA71538-17A3D8EEA-8C5-8D1': [{'com...
2,3,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6BEF87-75886535-93B54D73-8C5-880': [{'comm...
3,4,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6BE075-7EC43DAC-71F96BAA-8C5-91E': [{'comm...
4,5,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6BD03A-75884D11-716CEA51-8C5-91F': [{'comm...
